In [292]:
# Import libraries to use
import pandas as pd
import os
import datetime

In [293]:
# format right at the beginning how to view the float type
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

In [294]:
# --- Determining the path for each folder/file ---
raw_data_folder = '../../Tesla_Deliveries_Production/raw_data_'
processed_data_folder = '../../Tesla_Deliveries_Production/processed_data_'

data_stock = os.path.join(raw_data_folder,'tesla_stock.csv')
data_tesla = os.path.join(raw_data_folder,'tesla_results.csv')

Tesla_kaggle Dataframe

In [295]:
# Loading dataframe and take the opportunity to view crucial information
try:
    df_tesla = pd.read_csv(data_tesla)

    print('--- Informations about dataframe ---\n')
    df_tesla.info()

    print('\n--- Firsts lines ---\n')
    print(df_tesla.head())

except Exception as e:
    print(f'occurred a error on try load the file: {e}')

--- Informations about dataframe ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  2640 non-null   int64  
 1   Month                 2640 non-null   int64  
 2   Region                2640 non-null   object 
 3   Model                 2640 non-null   object 
 4   Estimated_Deliveries  2640 non-null   int64  
 5   Production_Units      2640 non-null   int64  
 6   Avg_Price_USD         2640 non-null   float64
 7   Battery_Capacity_kWh  2640 non-null   int64  
 8   Range_km              2640 non-null   int64  
 9   CO2_Saved_tons        2640 non-null   float64
 10  Source_Type           2640 non-null   object 
 11  Charging_Stations     2640 non-null   int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 247.6+ KB

--- Firsts lines ---

   Year  Month         Region    Model  Estimated_Deli

In [296]:
# adjusting the visualization of columns to make each one without whitespaces
df_tesla.columns = [column.strip() for column in df_tesla.columns]

# adjusting the format of columns to make each as 'Category'
columns_category = ['Region', 'Model', 'Source_Type']

for col in columns_category:
    df_tesla[col] = df_tesla[col].astype('category')

In [297]:
#Creating a new columns called 'Date', joining ['Year', 'Month'], and indexing it as index for the dataframe

df_tesla['Day'] = 1
df_tesla['Date'] = pd.to_datetime(df_tesla[['Year', 'Month', 'Day']])
df_tesla = df_tesla.drop(columns=['Day', 'Year', 'Month'])

df_tesla = df_tesla.set_index('Date')

print(df_tesla.head())

                   Region    Model  Estimated_Deliveries  Production_Units  \
Date                                                                         
2023-05-01         Europe  Model S                 17646             17922   
2015-02-01           Asia  Model X                  3797              4164   
2019-01-01  North America  Model X                  8411              9189   
2021-02-01  North America  Model 3                  6555              7311   
2016-12-01    Middle East  Model Y                 12374             13537   

            Avg_Price_USD  Battery_Capacity_kWh  Range_km  CO2_Saved_tons  \
Date                                                                        
2023-05-01      92,874.27                   120       704        1,863.42   
2015-02-01      62,205.65                    75       438          249.46   
2019-01-01     117,887.32                    82       480          605.59   
2021-02-01      89,294.91                   120       712          7

In [298]:
# Saving the dataframe, after my transformation and adjustment process

processed_file_path = os.path.join(processed_data_folder,'tesla_results.parquet')

df_tesla.to_parquet(processed_file_path)

print(f'Data transformed and saved in: {processed_file_path}')

Data transformed and saved in: ../../Tesla_Deliveries_Production/processed_data_\tesla_results.parquet


Stock Dataframe

In [299]:
# Loading dataframe and take the opportunity to view crucial information
try:
    df_stock = pd.read_csv(data_stock)

    print('--- Informations about dataframe ---\n')
    df_stock.info()

    print('\n--- Firsts lines ---\n')
    print(df_stock.head())

except Exception as e:
    print(f'occurred a error on try load the file: {e}')

--- Informations about dataframe ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2737 entries, 0 to 2736
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Price   2737 non-null   object
 1   Close   2736 non-null   object
 2   High    2736 non-null   object
 3   Low     2736 non-null   object
 4   Open    2736 non-null   object
 5   Volume  2736 non-null   object
dtypes: object(6)
memory usage: 128.4+ KB

--- Firsts lines ---

        Price               Close                High                 Low  \
0      Ticker                TSLA                TSLA                TSLA   
1        Date                 NaN                 NaN                 NaN   
2  2015-01-02  14.620667457580566  14.883333206176758   14.21733283996582   
3  2015-01-05  14.005999565124512  14.433333396911621  13.810667037963867   
4  2015-01-06  14.085332870483398  14.279999732971191   13.61400032043457   

                 Open    Volume  
0     

In [300]:
# Data Cleaning

df_stock_cleaned = df_stock.iloc[2:].copy()

df_stock_cleaned = df_stock_cleaned.rename(columns={'Price': 'Date'})

df_stock_cleaned['Date'] = pd.to_datetime(df_stock_cleaned['Date'])

df_stock_cleaned = df_stock_cleaned.set_index('Date')

df_stock_cleaned = df_stock_cleaned[['Close', 'Volume']]

cols_to_convert = ['Close','Volume']
df_stock_cleaned[cols_to_convert] = df_stock_cleaned[cols_to_convert].apply(pd.to_numeric)

df_stock_monthly = df_stock_cleaned.resample('MS').mean()

df_stock_monthly.index.name = None
df_stock_monthly.columns.name = 'Date'

df_stock_monthly

Date,Close,Volume
2015-01-01,13.55,"67,653,825.00"
2015-02-01,14.04,"84,250,657.89"
2015-03-01,12.98,"83,856,886.36"
2015-04-01,14.11,"70,675,071.43"
2015-05-01,16.15,"66,845,850.00"
...,...,...
2025-07-01,314.99,"98,064,031.82"
2025-08-01,331.14,"76,342,642.86"
2025-09-01,395.89,"93,981,052.38"
2025-10-01,441.33,"88,351,956.52"


In [301]:
# Saving the dataframe, after my transformation and adjustment process

processed_file_path = os.path.join(processed_data_folder,'tesla_stock.parquet')

df_stock_monthly.to_parquet(processed_file_path)

print(f'Data transformed and saved in: {processed_file_path}')

Data transformed and saved in: ../../Tesla_Deliveries_Production/processed_data_\tesla_stock.parquet
